In [1]:
import torch
from torchvision import models, transforms
from PIL import Image

In [ ]:
# Asumimos que tienes la arquitectura correcta de Wang2020, aquí uso ResNet solo como ejemplo
model = models.resnet50()  # Reemplazar con la arquitectura real de Wang2020
model.fc = torch.nn.Linear(model.fc.in_features, 2)  # Ajustar según la salida del modelo de Wang2020

In [ ]:
# Cargar los pesos del modelo
model.load_state_dict(torch.load('path/to/model_epoch_best.pth', map_location=torch.device('cpu')))
model.eval()

In [ ]:
# Preprocesamiento de la imagen
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Cargar la imagen
img_path = 'path/to/your/image.jpg'
img = Image.open(img_path)
img_tensor = preprocess(img).unsqueeze(0)  # Añade una dimensión para batch_size

In [ ]:
# Predicción
with torch.no_grad():
    outputs = model(img_tensor)
    # Convierte los logits a probabilidades usando softmax
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    probability_fake = probabilities[0][1].item()

print(f"Probabilidad de que la imagen sea artificial: {probability_fake * 100:.2f}%")